In [1]:
#Importamos las librerias
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import time
from numpy import linalg as LA
import random
from time import time
%matplotlib qt

In [2]:
#Definimos la funcion a minimizar
def f(x):
    fx = 100*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 + 90*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 -(20*x[0]+40*x[1])
    return fx

In [3]:
#Definir funcion para calcular la matriz del gradiente
def gradient(x,delta):
    grad=np.zeros(2)
    grad[0]=(f([x[0]+delta,x[1]])- f([x[0]-delta,x[1]]))/(2*delta)
    grad[1]=(f([x[0],x[1]+delta])- f([x[0],x[1]-delta]))/(2*delta)
    return grad

In [4]:
#Definir funcion para calcular la matriz Hessiana
def Hessian(x,delta):
    H=np.zeros([2,2])
    H[0,0]= (f([x[0]+delta,x[1]]) - 2*f([x[0],x[1]]) +  f([x[0]-delta,x[1]]))/(delta**2)
    H[1,1]= (f([x[0],x[1]+delta]) - 2*f([x[0],x[1]]) +  f([x[0],x[1]-delta]))/(delta**2)
    H[0,1]= (f([x[0]+delta,x[1]+delta]) - f([x[0]+delta,x[1]-delta]) - f([x[0]-delta,x[1]+delta]) +  f([x[0]-delta,x[1]-delta]))/(4*delta**2)
    H[1,0]= H[0,1]
    return H

In [5]:
#Utilizamos el algoritmo de golden section para encontrar alpha optimo
def golden(x,search,xi,eps):
    a = xi[0];
    b = xi[1];
    tau = 0.381967;
    alpha1 = a*(1-tau) + b*tau;
    alpha2 = a*tau + b*(1-tau);
    falpha1 = f(x+alpha1*search);
    falpha2 = f(x+alpha2*search);
    for i in range(100):
        if falpha1 > falpha2:
            a = alpha1;
            alpha1 = alpha2;
            falpha1 = falpha2;
            alpha2 = tau*a + (1-tau)*b;
            falpha2 = f(x+alpha2*search);
        else:
            b = alpha2;
            alpha2 = alpha1;
            falpha2 = falpha1;
            alpha1 = tau*b + (1-tau)*a;
            falpha1 = f(x+alpha1*search);
        if np.abs(f(x+alpha1*search)- f(x+alpha2*search)) < eps :
            break;
    return alpha1,falpha1

In [6]:
#Graficamos la funcion de prueba
X1 = np.arange(-5, 5, 0.01)
X2 = np.arange(-5, 5, 0.01)
x1, x2 = np.meshgrid(X1, X2)
z = f([x1,x2])
fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
surf = ax.plot_surface(x1, x2, z, cmap=cm.autumn, linewidth=0, antialiased = False, alpha=0.2)
plt.show()

In [7]:
#Variable en comun de los métodos
delta = 1e-3 
ep1 = 1e-3
puntosFB = []

In [10]:
def graficaUnaaUna(puntos, nombreMetodo):
    #Generamos valores ramdom para generar un color diferente de grafica
    r = random.random()
    b = random.random()
    g = random.random()
    color = (r, g, b)
    aux = 0
    x = np.linspace(-1, 1, 100)
    y = np.linspace(-1, 1, 100)
    xx, yy = np.meshgrid(x, y)
    z = f([xx, yy])
    plt.contour(x,y,z,20)
    plt.plot(-1,-1, 'ro--',  markersize=6)
    plt.plot(0.5,0, 'go--', markersize=6)
    #Graficamos los trazos (direcciones) que va generando el algoritmo
    for pts in puntos:
        if(aux==len(puntos)-1):
            plt.plot(pts[0],pts[1], c=color, linewidth=2, label=str(nombreMetodo))
        else:
            plt.plot(pts[0],pts[1], c=color, linewidth=2)
        aux = aux + 1
    plt.legend()    
    plt.show

In [11]:
start_time = time()
xi_fb = [-1,1] #Punto de partida
xa_fb =  xi_fb
alpha = 0.001
iteraciones = 0
fx_value = f(xi_fb) #Se evalua la funcion con el punto inicial
print('Valor de la función inicial = %f ' % fx_value)
for i in range(1000): #Proceso iterativo del algoritmo
    xa_fb =  xi_fb
    si = gradient(xi_fb, delta)
    s1 = -1*si[0]
    s2 = -1*si[1]
    xdd = xi_fb[0] + alpha*s1
    ydd = xi_fb[1] + alpha*s2
    fdd1 = f([xdd, ydd])
    fdd2 = f(xi_fb)
    if(abs(fdd1-fdd2)<=ep1):
        print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, xi_fb[0], xi_fb[1], f(xi_fb), LA.norm(si)))
        break
    xi_fb = [xdd, ydd]
    print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, xi_fb[0], xi_fb[1], f(xi_fb), LA.norm(si)))
    puntosFB.append([[xa_fb[0],xi_fb[0]],[xa_fb[1],xi_fb[1]]]) #Almacenamos los punto de direccion del algoritmo
    iteraciones += 1
elapsed_time = (time() - start_time)*1000
print("Tiempo transcurrido: %f" % elapsed_time)
print("Iteraciones: %d" % iteraciones)
#Graficamos el resultado del Steepest descent method
graficaUnaaUna(puntosFB, 'Brute Force')

Valor de la función inicial = 270.294169 
0	[-0.7699,0.6199]		110.2512		444.3158
1	[-0.6205,0.3875]		48.2750		276.2647
2	[-0.5198,0.2472]		23.9401		172.7293
3	[-0.4485,0.1640]		14.0401		109.5750
4	[-0.3947,0.1163]		9.6750		71.9210
5	[-0.3514,0.0903]		7.4400		50.4692
6	[-0.3144,0.0777]		6.0375		39.0719
7	[-0.2814,0.0730]		4.9762		33.3881
8	[-0.2508,0.0728]		4.0712		30.5303
9	[-0.2220,0.0752]		3.2533		28.8984
10	[-0.1945,0.0787]		2.4967		27.7526
11	[-0.1680,0.0828]		1.7909		26.7903
12	[-0.1425,0.0868]		1.1314		25.8974
13	[-0.1177,0.0906]		0.5152		25.0343
14	[-0.0937,0.0940]		-0.0599		24.1892
15	[-0.0706,0.0969]		-0.5961		23.3599
16	[-0.0482,0.0993]		-1.0955		22.5472
17	[-0.0265,0.1012]		-1.5602		21.7524
18	[-0.0056,0.1026]		-1.9923		20.9767
19	[0.0146,0.1036]		-2.3938		20.2211
20	[0.0341,0.1042]		-2.7665		19.4860
21	[0.0529,0.1044]		-3.1124		18.7720
22	[0.0710,0.1043]		-3.4332		18.0792
23	[0.0884,0.1039]		-3.7305		17.4074
24	[0.1051,0.1032]		-4.0060		16.7568
25	[0.1212,0.1023]		-4.2612		